## Materials 
At the start, we have audio and annotated textgrids of **regilaul** songs, annotated for ictus/off-ictus and phrase text, then force-aligned using Praat's built in eSpeak forced aligner for Estonian to word and then segment. Then, we use the estnltk vabamorf package to syllabify the words so that we can annotate the textgrid further with syllable quantity (Estonian has 3) and whether or not it is accented at the word level. We end up with a dataframe containing the data from three of the(Interval) tiers of the textgrid, acquiring duration data for words, individual segments, and (eventually) syllables. 

In [1]:
import pandas as pd
import parselmouth
from estnltk.vabamorf.morf import syllabify_word
import tgt
import string

#test method on a single TextGrid:
gridDir2 = "/Users/sarah/Git/qp_final/songs/txtgridtest/09.TextGrid"


def get_duration_labels(textgrid, wordTier,s1,s2,ictusTier):
    #tmp = codecs.open(textgrid,'r','utf-8')
    tmp = tgt.io.read_textgrid(textgrid)
    words = tmp.get_tier_by_name(wordTier)
    firstSyll = tmp.get_tier_by_name(s1)
    secSyll = tmp.get_tier_by_name(s2)
    ictus = tmp.get_tier_by_name(ictusTier)
    segments = []
    wordlist = words.intervals
    
    for interval in wordlist:
        onset = interval.start_time
        offset = interval.end_time
       # wordms = offset-onset
        word = interval.text
        syllablist = syllabify_word(word,as_dict=True)

        i = 0
        #keep less than length of the list to avoid indexing errors on monosyllabic words
        
        while i < len(syllablist): 
            tmpsy = syllablist[i] 
            ortho = tmpsy.get('syllable')
            q = tmpsy.get('quantity')
            a = tmpsy.get('accent')

            if i == 0: 
                tmpinterval = firstSyll.get_annotations_between_timepoints(onset,offset)
            elif i == 1: 
                tmpinterval = secSyll.get_annotations_between_timepoints(onset,offset)
            #break out of the loop after getting the second syllable
        
            #skip syllables with no annotations in the analysis tiers
            if len(tmpinterval)==0: 
                i+= 1 
                break
            for vowel in tmpinterval:
              
                segment = vowel.text
                vOnset = vowel.start_time
                vOffset = vowel.end_time
                vMidpoint = vOffset - (vOnset/2)
                dur = vOffset-vOnset
                tmpick = ictus.get_annotations_by_time(vMidpoint)
                if len(tmpick) > 0 :
                    ick = tmpick[0].text
                else: ick = "off"
                row = (word,ortho,i,segment,q,a,ick,dur,vMidpoint) 
                segments.append(row)
                
            i+= 1  
            
    nu_df = pd.DataFrame(segments,columns=["word","syll","index","segment","quantity","stressed","ictus","duration","midpoint"])
    return nu_df
    

# syl_dur_df = get_duration_labels(gridDir2,"word","word/phon","ictus")
# syl_dur_df.head()

onetwo_df = get_duration_labels(gridDir2,"word","s1","s2","ictus")
onetwo_df



/Users/sarah/miniconda3/envs/soundmines/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,word,syll,index,segment,quantity,stressed,ictus,duration,midpoint
0,"Lõpe,",lõ,0,ɵ,2,1,ictus,0.221401,0.365285
1,"lõpe,",lõ,0,ɵ,2,1,x,0.208552,0.770125
2,"lõpe,","pe,",1,e,2,0,ictus,0.293176,1.077015
3,"linakene,",li,0,i,1,1,ictus,0.222958,1.189854
4,"linakene,",na,1,ɑ,2,0,x,0.312941,1.424412
5,kui,kui,0,uiː,3,1,ictus,0.382281,3.018528
6,sa,sa,0,a,3,1,ictus,0.165045,3.046895
7,"lõpe,",lõ,0,ɵ,2,1,x,0.216827,3.277663
8,siia,sii,0,iː,2,1,refrain,0.287818,3.745161
9,siia,a,1,ja,1,0,refrain,0.373816,3.975067


## Adding Spectral data
now that we have the duration data from the textgrid, we can query specific timepoints for information about the acoustic signal. The following function uses the midpoint (which we snagged while we were making the dataframe above) and get the first three formants(Hz) for each segment. 

In [19]:

import parselmouth

test = "songs/matchgrids/65.wav"

def get_formants(syl_dur_df, wave):
    song = parselmouth.Sound(wave)
    formant = song.to_formant_burg()
    f1 = []
    f2 = []
    for float in syl_dur_df.midpoint:
        time = float
        first = formant.get_value_at_time(1,time)
        f1.append(first)
        second = formant.get_value_at_time(2, time)
        f2.append(second)
    syl_dur_df["f1"] = f1
    syl_dur_df["f2"] = f2
    return syl_dur_df
nu_df = get_formants(onetwo_df,test)
nu_df.head()


,word,syll,index,segment,quantity,stressed,ictus,duration,midpoint,f1,f2
0,"Lõpe,",lõ,0,ɵ,2,1,ictus,0.221401,0.365285,549.684786,2604.852094
1,"lõpe,",lõ,0,ɵ,2,1,x,0.208552,0.770125,801.339195,1357.104620
2,"lõpe,","pe,",1,e,2,0,ictus,0.293176,1.077015,596.245917,1029.855239
3,"linakene,",li,0,i,1,1,ictus,0.222958,1.189854,385.956221,1069.491806
4,"linakene,",na,1,ɑ,2,0,x,0.312941,1.424412,397.151510,1056.330500


In [20]:
from os.path import join
#runs a for loop over a directory using the above-specified functions

test = "songs/txtgridtest"
songs = "songs/matchgrids"

for fn in os.listdir(test):
    if '.TextGrid' not in fn: 
        continue 
    n = fn.strip('.TextGrid') 
    wave = join(songs, n + '.wav')
    data_file = open(  n +"_updated.csv",'w')
    #make a dataframe with the interval tiers of the textgrid
    tmp = pd.DataFrame(get_duration_labels(join(test,fn), "word","s1","s2","ictus"))
    #add the formant data to the dataframe
    nu_df = get_formants(tmp,wave)
    #print(nu_df.head())
    nu_df.to_csv(data_file)
    data_file.close()

# Now we put it into a big pile!

Here we concatenate all the data we have so far into one large pandas dataframe. At this point, we can keep annotating songs for the corpus, and as textgrids are finished we can run the scripts above to add them into the larger dataset. We're also gonna take the opportunity to add some metadata to the dataframes: fileid(song) and performer initials as potential grouping factors. 

In [12]:

import os 
import pandas as pd 
import statsmodels.formula.api as smf
folder = "updated"
meta =  pd.read_csv("songs/song_metadata.csv")


songs_dfs = []
for fn in os.listdir(folder):
    if '.csv' not in fn: continue
    whole_name = os.path.join(folder,fn)
    song_df = pd.read_csv(whole_name)
    fileid1 = fn.strip('_updated.csv')
    fileid = int(fileid1)
    row = meta.index[meta['track'] == fileid].tolist()
    if len(row) !=0 :
        performer = meta.performer[row[0]]
    else: performer = "couldn't get match"
    for index in song_df:
        song_df['fileid'] = fileid
        song_df['performer'] = performer

        
    songs_dfs.append(song_df)

big_frame = pd.concat(songs_dfs, ignore_index=True)
big_frame.describe()

# big_frame
#clean_frame = pd.DataFrame(big_frame[['quantity','stress','segment','seg_duration','ictus','euc','fileid','performer']])
#clean_frame.head()

big_frame

,Unnamed: 0,word,syll,index,segment,quantity,stressed,ictus,duration,midpoint,f1,f2,fileid,performer
0,0,"Laula,",lau,0,au,2,1,ictus,0.288020,0.522161,523.054476,959.090344,55,LO
1,1,"Laula,","la,",1,a,2,0,ictus,0.150032,0.569267,577.399028,1042.898541,55,LO
2,2,"laula,",lau,0,au,2,1,x,0.228340,0.762019,628.070156,1613.323607,55,LO
3,3,"laula,","la,",1,a,2,0,x,0.121089,0.813915,367.311972,1389.969470,55,LO
4,4,"suukene,",suu,0,uː,2,1,ictus,0.359808,1.196289,525.125022,1081.622277,55,LO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,64,"jalgas,","gas,",1,a,2,0,ictus,0.105363,19.878965,789.750978,2194.735989,69,LO
646,65,kuusekoorest,kuu,0,uː,2,1,ictus,0.202407,20.959403,920.767293,1731.370866,69,LO
647,66,kuusekoorest,se,1,e,1,0,ictus,0.095658,21.010698,961.346457,1960.120839,69,LO
648,67,selgas.,sel,0,el,2,1,x,0.211360,21.764951,508.474407,2135.156482,69,LO


In [13]:
big_frame['index'] = big_frame['index'].astype(object)
big_frame['quantity'] = big_frame['quantity'].astype(object)
big_frame['stressed'] = big_frame['stressed'].astype(object)
big_frame['fileid'] = big_frame['fileid'].astype(object)

corpus_data = open('bu_all_songs.csv','w')
big_frame.to_csv(corpus_data)
corpus_data.close()
big_frame.dtypes

Unnamed: 0      int64
word           object
syll           object
index          object
segment        object
quantity       object
stressed       object
ictus          object
duration      float64
midpoint      float64
f1            float64
f2            float64
fileid         object
performer      object
dtype: object

In [7]:
big_frame.groupby('stressed')['duration'].describe()

,count,mean,std,min,25%,50%,75%,max
stressed,,,,,,,,
0,271.0,0.186972,0.071192,0.035638,0.138025,0.182604,0.227726,0.412773
1,339.0,0.218175,0.097679,0.025764,0.148371,0.206735,0.277601,0.570958
